## Language Translator

In [11]:
import nltk
import collections
#import helper
import numpy as np
#import project_tests as tests
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [12]:
import joblib
from tensorflow.keras.models import Sequential

From `nltk` we can download translated sentences between different languages. You can see the example between **English and French** below but feel free to try different combination as well.

In [ ]:
nltk.download('comtrans')

[nltk_data] Downloading package comtrans to
[nltk_data]     C:\Users\huawei\AppData\Roaming\nltk_data...


True

In [ ]:
from nltk.corpus import comtrans
print(comtrans.aligned_sents('alignment-en-fr.txt')[0])

<AlignedSent: 'Resumption of the se...' -> 'Reprise de la sessio...'>


In [ ]:
len(comtrans.aligned_sents('alignment-en-fr.txt'))

33334

In [ ]:
eng_text = []
frn_text = []
for i in range(334):
    als = comtrans.aligned_sents("alignment-en-fr.txt")[i]
    eng_text.append(" ".join(als.words))
    frn_text.append(" ".join(als.mots))
print(len(eng_text))
print(len(frn_text))

334
334


In [13]:
frn_text = joblib.load('frn_text.joblib')
eng_text = joblib.load('eng_text.joblib')

In [42]:
eng_text[18]

'This is all in accordance with the principles that we have always upheld .'

In [14]:
## Tokenzie
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [15]:
##padding
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

In [16]:
## combine preparcessing
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(eng_text)
    preprocess_y, y_tk = tokenize(frn_text)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    #preprocess_x = preprocess_x.reshape(*preprocess_x.shape, 1)
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    
    return preprocess_x, preprocess_y, x_tk, y_tk

In [17]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(eng_text, frn_text)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 41
Max French sentence length: 41
English vocabulary size: 15553
French vocabulary size: 21326


In [ ]:
preproc_english_sentences.shape

(31500, 41)

In [18]:
## from ids to words
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    print('`logits_to_text` function loaded.')
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])


In [ ]:
## building models
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = 'sparse_categorical_crossentropy', 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

#tests.test_simple_model(simple_model)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=50, epochs=10, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
6/6 [==============================] - 1s 250ms/step - loss: 7.3790 - accuracy: 0.3301 - val_loss: 7.3010 - val_accuracy: 0.3920
Epoch 2/10
6/6 [==============================] - 1s 142ms/step - loss: 7.2434 - accuracy: 0.4830 - val_loss: 7.1744 - val_accuracy: 0.4101
Epoch 3/10
6/6 [==============================] - 1s 180ms/step - loss: 7.0917 - accuracy: 0.4898 - val_loss: 7.0117 - val_accuracy: 0.4108
Epoch 4/10
6/6 [==============================] - 1s 145ms/step - loss: 6.8810 - accuracy: 0.4894 - val_loss: 6.7507 - val_accuracy: 0.4081
Epoch 5/10
6/6 [==============================] - 1s 140ms/step - loss: 6.4844 - accuracy: 0.4904 - val_loss: 6.2135 - val_accuracy: 0.4124
Epoch 6/10
6/6 [==============================] - 1s 132ms/step - loss: 5.7724 - accuracy: 0.4925 - val_loss: 5.6903 - val_accuracy: 0.4179
Epoch 7/10
6/6 [==============================] - 1s 173ms/step - loss: 5.2103 - accuracy: 0.4976 - val_loss: 5.2998 - val_accuracy: 0.4226
Epoch 8/10
6/6 [====

In [ ]:
from tensorflow.keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
#tests.test_embed_model(embed_model)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=50, epochs=10, validation_split=0.2)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
6/6 [==============================] - 1s 218ms/step - loss: 7.3983 - accuracy: 0.3123 - val_loss: 7.3751 - val_accuracy: 0.4273
Epoch 2/10
6/6 [==============================] - 1s 98ms/step - loss: 7.3425 - accuracy: 0.5060 - val_loss: 7.3043 - val_accuracy: 0.4277
Epoch 3/10
6/6 [==============================] - 1s 105ms/step - loss: 7.2271 - accuracy: 0.5059 - val_loss: 7.1303 - val_accuracy: 0.4277
Epoch 4/10
6/6 [==============================] - 1s 96ms/step - loss: 6.9096 - accuracy: 0.5059 - val_loss: 6.5676 - val_accuracy: 0.4277
Epoch 5/10
6/6 [==============================] - 1s 104ms/step - loss: 5.9510 - accuracy: 0.5059 - val_loss: 5.4200 - val_accuracy: 0.4277
Epoch 6/10
6/6 [==============================] - 1s 103ms/step - loss: 4.8380 - accuracy: 0.5059 - val_loss: 4.7594 - val_accuracy: 0.4277
Epoch 7/10
6/6 [==============================] - 1s 119ms/step - loss: 4.2139 - accuracy: 0.5059 - val_loss: 4.4302 - val_accuracy: 0.4277
Epoch 8/10
6/6 [======

In [25]:
## Model 3
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
#tests.test_bd_model(bd_model)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1000, epochs=20, validation_split=0.2)
#joblib.dump(bidi_model,'bidimodel.joblib')
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
26/26 [==============================] - 122s 5s/step - loss: 8.1497 - accuracy: 0.3447 - val_loss: 5.1426 - val_accuracy: 0.5164
Epoch 2/20
26/26 [==============================] - 115s 4s/step - loss: 4.2973 - accuracy: 0.5068 - val_loss: 3.9316 - val_accuracy: 0.5137
Epoch 3/20
26/26 [==============================] - 117s 5s/step - loss: 3.8195 - accuracy: 0.5067 - val_loss: 3.6780 - val_accuracy: 0.5137
Epoch 4/20
26/26 [==============================] - 117s 4s/step - loss: 3.6438 - accuracy: 0.5069 - val_loss: 3.5705 - val_accuracy: 0.5145
Epoch 5/20
26/26 [==============================] - 116s 4s/step - loss: 3.5645 - accuracy: 0.5087 - val_loss: 3.5278 - val_accuracy: 0.5187
Epoch 6/20
26/26 [==============================] - 118s 5s/step - loss: 3.5294 - accuracy: 0.5118 - val_loss: 3.5118 - val_accuracy: 0.5225
Epoch 7/20
26/26 [==============================] - 115s 4s/step - loss: 3.5116 - accuracy: 0.5143 - val_loss: 3.5004 - val_accuracy: 0.5231
Epoch 8/20
26

KeyboardInterrupt: ignored

In [29]:
print(logits_to_text(bidi_model.predict(tmp_x[:5])[0], french_tokenizer))

`logits_to_text` function loaded.
de de <PAD> de <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [ ]:
## Model4
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
#tests.test_encdec_model(encdec_model)
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=50, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/20
6/6 [==============================] - 2s 328ms/step - loss: 7.3923 - accuracy: 0.4091 - val_loss: 7.3319 - val_accuracy: 0.4277
Epoch 2/20
6/6 [==============================] - 1s 156ms/step - loss: 7.1805 - accuracy: 0.5059 - val_loss: 6.5681 - val_accuracy: 0.4277
Epoch 3/20
6/6 [==============================] - 1s 176ms/step - loss: 5.4024 - accuracy: 0.5059 - val_loss: 4.5671 - val_accuracy: 0.4277
Epoch 4/20
6/6 [==============================] - 1s 152ms/step - loss: 3.9294 - accuracy: 0.5059 - val_loss: 4.3088 - val_accuracy: 0.4277
Epoch 5/20
6/6 [==============================] - 1s 193ms/step - loss: 3.6809 - accuracy: 0.5059 - val_loss: 4.3224 - val_accuracy: 0.4277
Epoch 6/20
6/6 [==============================] - 1s 159ms/step - loss: 3.6264 - accuracy: 0.5059 - val_loss: 4.3232 - val_accuracy: 0.4277
Epoch 7/20
6/6 [==============================] - 1s 201ms/step - loss: 3.5956 - accuracy: 0.5059 - val_loss: 4.3104 - val_accuracy: 0.4277
Epoch 8/20
6/6 [====

In [33]:
## Model 5 combine 3 and 4
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
#tests.test_model_final(model_final)
print('Final Model Loaded')

Final Model Loaded


In [35]:
#def final_predictions(x, y, x_tk, y_tk):
    
tmp_X = pad(preproc_english_sentences)
model = model_final(tmp_X.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
model.fit(tmp_X, preproc_french_sentences, batch_size = 500, epochs = 18, validation_split = 0.2)

Epoch 1/18
51/51 [==============================] - 196s 4s/step - loss: 4.3432 - accuracy: 0.4967 - val_loss: 3.7998 - val_accuracy: 0.5180
Epoch 2/18
51/51 [==============================] - 190s 4s/step - loss: 3.7337 - accuracy: 0.5142 - val_loss: 3.6736 - val_accuracy: 0.5281
Epoch 3/18
51/51 [==============================] - 189s 4s/step - loss: 3.6565 - accuracy: 0.5202 - val_loss: 3.6996 - val_accuracy: 0.5273
Epoch 4/18
51/51 [==============================] - 187s 4s/step - loss: 3.6408 - accuracy: 0.5215 - val_loss: 3.6345 - val_accuracy: 0.5276
Epoch 5/18
51/51 [==============================] - 191s 4s/step - loss: 3.6212 - accuracy: 0.5224 - val_loss: 3.6349 - val_accuracy: 0.5284
Epoch 6/18
51/51 [==============================] - 190s 4s/step - loss: 3.6170 - accuracy: 0.5230 - val_loss: 3.6373 - val_accuracy: 0.5292
Epoch 7/18
51/51 [==============================] - 191s 4s/step - loss: 3.5962 - accuracy: 0.5242 - val_loss: 3.6537 - val_accuracy: 0.5291
Epoch 8/18
51

In [44]:
model.fit(tmp_X, preproc_french_sentences, batch_size = 64, epochs = 3, validation_split = 0.2)

Epoch 1/3
394/394 [==============================] - 359s 912ms/step - loss: 3.8675 - accuracy: 0.5331 - val_loss: 3.9895 - val_accuracy: 0.5366
Epoch 2/3
394/394 [==============================] - 348s 883ms/step - loss: 3.8906 - accuracy: 0.5355 - val_loss: 3.9832 - val_accuracy: 0.5381
Epoch 3/3
394/394 [==============================] - 337s 857ms/step - loss: 3.8609 - accuracy: 0.5371 - val_loss: 3.9913 - val_accuracy: 0.5349


In [45]:

 
y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
y_id_to_word[0] = '<PAD>'
sentence = ' vote for the   is like a vote for the Liberals'
sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
sentences = np.array([sentence[0], preproc_english_sentences[0]])
predictions = model.predict(sentences, len(sentences))
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Il a vu un vieux camion jaune')
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in preproc_french_sentences[0]]))

    
#final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

KeyError: ignored